## Librerias

In [24]:
import pymongo
from pymongo import MongoClient
import geopandas as gpd
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json

from functools import reduce
import operator

load_dotenv()

import sys
sys.path.append('../')
import src.Fapis as fap

## Importacion de datos

In [4]:
oficina = pd.read_csv('../data/companies.csv',encoding = 'ISO - 8859-1')


In [5]:
dfoficinas = pd.DataFrame(oficina)
dfoficinas

,city,latitude,longitude,name,category_code,number_of_employees,founded_year
0,Madrid,40.530724,-3.632687,Fon,mobile,100,2006
1,Madrid,40.415824,-3.696277,Tuenti Technologies,social,170,2006


In [7]:
# comprobacion de geoenterprise
company_location = fap.geoenterprise(dfoficinas)
company_location

{'Fon': {'type': 'Point', 'coordinates': [40.530724, -3.632687]},
 'Tuenti Technologies': {'type': 'Point',
  'coordinates': [40.4158236, -3.696277]}}

## LLamadas a API

### tuenti

In [51]:
la = company_location['Tuenti Technologies']['coordinates'][0]
lg = company_location['Tuenti Technologies']['coordinates'][1]

###### Starbucks

In [52]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2") 
url_query = 'https://api.foursquare.com/v2/venues/explore'

query = 'starbucks'
parametros = {'client_id':tok1,
              'client_secret':tok2,
              'v':'20180323',
              'll':f'{la},{lg}',
              'query': query,
              'limit':50
             }
# Generamos Request 
resp = requests.get(url=url_query,params=parametros)
data =json.loads(resp.text)


In [53]:
############### DECODING ###########
decoding_data = data.get("response")
decoded = decoding_data.get('groups')[0]
starbucks =decoded.get('items')

In [54]:
# creacion de DF con los datos del json
mapa_nombre =['venue','name']
m_latitud =['venue','location','lat']
m_longitud = ['venue','location','lng']

In [55]:
# introducimos los datos de json en DF 
lovemosclaro = []
for dic in starbucks:
    paralista ={}
    paralista['enterprise'] = 'Tuenti Technologies'
    paralista['team'] = 'Executives'
    paralista['requirements'] = 'starbucks'
    paralista['name'] = getFromDict(dic,mapa_nombre)
    paralista['latitud'] = getFromDict(dic,m_latitud)
    paralista['longitud'] = getFromDict(dic,m_longitud)
    lovemosclaro.append(paralista)


In [80]:
#creamos un data frame con lista de dicionarios si y solo si siempre son las mismas key y mismos values
gdf_starbucks_1 =pd.DataFrame(lovemosclaro)
gdf_starbucks_1[:3]

,enterprise,team,requirements,name,latitud,longitud
0,Fon,Account,Airport Terminal,T4 Salidas,40.406544,-3.682036


######  Museos

In [57]:
query = 'Museum'
parametros = {'client_id':tok1,
              'client_secret':tok2,
              'v':'20180323',
              'll':f'{la},{lg}',
              'query': query,
              'limit':50
             }
# Generamos Request 
resp = requests.get(url=url_query,params=parametros)
data =json.loads(resp.text)
############### DECODING ###########
decoding_data = data.get("response")
decoded = decoding_data.get('groups')[0]
museum =decoded.get('items')
# introducimos los datos de json en DF 
lovemosclaro = []
for dic in museum:
    paralista ={}
    paralista['enterprise'] = 'Tuenti Technologies'
    paralista['team'] = 'Designers'
    paralista['requirements'] = 'Museum'
    paralista['name'] = getFromDict(dic,mapa_nombre)
    paralista['latitud'] = getFromDict(dic,m_latitud)
    paralista['longitud'] = getFromDict(dic,m_longitud)
    lovemosclaro.append(paralista)
#creamos un data frame con lista de dicionarios si y solo si siempre son las mismas key y mismos values
gdf_museum_1 =pd.DataFrame(lovemosclaro)

'# gpd.points_from_xy esto siempre stiene que estar escrito asi \ngdf_museum_1 = gpd.GeoDataFrame(dataframefinal_1,geometry =gpd.points_from_xy(dataframefinal_1.longitud,dataframefinal_1.latitud))\ngdf_museum_1'

In [81]:
gdf_museum_1[:3]

,enterprise,team,requirements,name,latitud,longitud
0,Tuenti Technologies,Designers,Museum,Museo Thyssen-Bornemisza,40.416359,-3.694891
1,Tuenti Technologies,Designers,Museum,Casa Museo Lope de Vega,40.414326,-3.697468
2,Tuenti Technologies,Designers,Museum,Círculo de Bellas Artes,40.418486,-3.696612


######  Aereopuerto

In [82]:
query = 'Airport Terminal'
parametros = {'client_id':tok1,
              'client_secret':tok2,
              'v':'20180323',
              'll':f'{la},{lg}',
              'query': query,
              'limit':50
             }
# Generamos Request 
resp = requests.get(url=url_query,params=parametros)
data =json.loads(resp.text)
############### DECODING ###########
decoding_data = data.get("response")
decoded = decoding_data.get('groups')[0]
airport =decoded.get('items')
# introducimos los datos de json en DF 
lovemosclaro = []
for dic in airport:
    paralista ={}
    paralista['enterprise'] = 'Tuenti Technologies'
    paralista['team'] = 'Account'
    paralista['requirements'] = 'Airport Terminal'
    paralista['name'] = getFromDict(dic,mapa_nombre)
    paralista['latitud'] = getFromDict(dic,m_latitud)
    paralista['longitud'] = getFromDict(dic,m_longitud)
    lovemosclaro.append(paralista)
#creamos un data frame con lista de dicionarios si y solo si siempre son las mismas key y mismos values
gdf_airport_1 =pd.DataFrame(lovemosclaro)

In [83]:
dfFinal_Tuenti = pd.concat([gdf_starbucks_1 , gdf_museum_1, gdf_airport_1])

In [84]:
dfFinal_Tuenti[:3]

,enterprise,team,requirements,name,latitud,longitud
0,Fon,Account,Airport Terminal,T4 Salidas,40.406544,-3.682036
0,Tuenti Technologies,Designers,Museum,Museo Thyssen-Bornemisza,40.416359,-3.694891
1,Tuenti Technologies,Designers,Museum,Casa Museo Lope de Vega,40.414326,-3.697468


### Fon

In [61]:
# introducimos las coordenadas de las empresas PARA FON 
laf = company_location['Tuenti Technologies']['coordinates'][0]
lgf = company_location['Tuenti Technologies']['coordinates'][1]


###### Starbucks

In [72]:
query = 'starbucks'
parametros = {'client_id':tok1,
              'client_secret':tok2,
              'v':'20180323',
              'll':f'{laf},{lgf}',
              'query': query,
              'limit':50
             }
# Generamos Request 
resp = requests.get(url=url_query,params=parametros)
data =json.loads(resp.text)
############### DECODING ###########
decoding_data = data.get("response")
decoded = decoding_data.get('groups')[0]
starbucks =decoded.get('items')
# introducimos los datos de json en DF 
lovemosclaro = []
for dic in starbucks:
    paralista ={}
    paralista['enterprise'] = 'Fon'
    paralista['team'] = 'Executives'
    paralista['requirements'] = 'starbucks'
    paralista['name'] = getFromDict(dic,mapa_nombre)
    paralista['latitud'] = getFromDict(dic,m_latitud)
    paralista['longitud'] = getFromDict(dic,m_longitud)
    lovemosclaro.append(paralista)
#creamos un data frame con lista de dicionarios si y solo si siempre son las mismas key y mismos values
gdf_starbucks_2 =pd.DataFrame(lovemosclaro)

'# gpd.points_from_xy esto siempre stiene que estar escrito asi \ngdf_starbucks_2 = gpd.GeoDataFrame(dataframefinal_2,geometry =gpd.points_from_xy(dataframefinal_2.longitud,dataframefinal_2.latitud))\ngdf_starbucks_2'

###### Museos

In [73]:
query = 'Museum'
parametros = {'client_id':tok1,
              'client_secret':tok2,
              'v':'20180323',
              'll':f'{laf},{lgf}',
              'query': query,
              'limit':50
             }
# Generamos Request 
resp = requests.get(url=url_query,params=parametros)
data =json.loads(resp.text)
############### DECODING ###########
decoding_data = data.get("response")
decoded = decoding_data.get('groups')[0]
museum =decoded.get('items')
# introducimos los datos de json en DF 
lovemosclaro = []
for dic in museum:
    paralista ={}
    paralista['enterprise'] = 'Fon'
    paralista['team'] = 'Designers'
    paralista['requirements'] = 'Museum'
    paralista['name'] = getFromDict(dic,mapa_nombre)
    paralista['latitud'] = getFromDict(dic,m_latitud)
    paralista['longitud'] = getFromDict(dic,m_longitud)
    lovemosclaro.append(paralista)
#creamos un data frame con lista de dicionarios si y solo si siempre son las mismas key y mismos values
gdf_museum_2 =pd.DataFrame(lovemosclaro)

'# gpd.points_from_xy esto siempre stiene que estar escrito asi \ngdf_museum_2 = gpd.GeoDataFrame(dataframefinal_2,geometry =gpd.points_from_xy(dataframefinal_2.longitud,dataframefinal_2.latitud))\ngdf_museum_2'

###### Aereopuerto

In [74]:
query = 'Airport Terminal'
parametros = {'client_id':tok1,
              'client_secret':tok2,
              'v':'20180323',
              'll':f'{laf},{lgf}',
              'query': query,
              'limit':50
             }
# Generamos Request 
resp = requests.get(url=url_query,params=parametros)
data =json.loads(resp.text)
############### DECODING ###########
decoding_data = data.get("response")
decoded = decoding_data.get('groups')[0]
airport =decoded.get('items')
# introducimos los datos de json en DF 
lovemosclaro = []
for dic in airport:
    paralista ={}
    paralista['enterprise'] = 'Fon'
    paralista['team'] = 'Account'
    paralista['requirements'] = 'Airport Terminal'
    paralista['name'] = getFromDict(dic,mapa_nombre)
    paralista['latitud'] = getFromDict(dic,m_latitud)
    paralista['longitud'] = getFromDict(dic,m_longitud)
    lovemosclaro.append(paralista)
#creamos un data frame con lista de dicionarios si y solo si siempre son las mismas key y mismos values
gdf_airport_2 =pd.DataFrame(lovemosclaro)

'# gpd.points_from_xy esto siempre stiene que estar escrito asi \ngdf_airport_2 = gpd.GeoDataFrame(dataframefinal_2,geometry =gpd.points_from_xy(dataframefinal_2.longitud,dataframefinal_2.latitud))\ngdf_airport_2'

In [86]:
dfFinal_Fon= pd.concat([gdf_starbucks_2 , gdf_museum_2, gdf_airport_2])
dfFinal_Fon[:3] 

,enterprise,team,requirements,name,latitud,longitud
0,Fon,Executives,starbucks,Starbucks,40.415323,-3.694949
1,Fon,Executives,starbucks,Starbucks,40.418038,-3.699372
2,Fon,Executives,starbucks,Starbucks,40.420752,-3.699315


In [87]:
dfFinal = pd.concat([dfFinal_Fon , dfFinal_Tuenti])
dfFinal[:3]

,enterprise,team,requirements,name,latitud,longitud
0,Fon,Executives,starbucks,Starbucks,40.415323,-3.694949
1,Fon,Executives,starbucks,Starbucks,40.418038,-3.699372
2,Fon,Executives,starbucks,Starbucks,40.420752,-3.699315


## Exportacion de datos

In [77]:
dfFinal.to_csv("../data/final.csv", index = False)

In [78]:
dfFinal_Fon.to_csv("../data/Fon.csv", index = False)

In [79]:
dfFinal_Tuenti.to_csv("../data/Tuenti.csv", index = False)